In [ ]:
# Import libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Cropping2D
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Softmax
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
import xml.etree.ElementTree as ET
import os
import cv2
import math
import pickle as pkl
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.heatmaps import HeatmapsOnImage
from tensorflow.python.client import device_lib
%matplotlib notebook
import matplotlib.pyplot as plt
import time
from google.colab.patches import cv2_imshow

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

print(tf.__version__)
print(tf.test.is_gpu_available()) # True/False

# Or only check for gpu's with cuda support
print(tf.test.is_gpu_available(cuda_only=True))
# print(device_lib.list_local_devices())
# K.tensorflow_backend._get_available_gpus()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


1 Physical GPUs, 1 Logical GPUs
2.17.0
True
True


In [ ]:
ANNOT = "..\\FIGC\\Annotation\\Film Role-0 ID-{} T-0 m00s00-026-m00s01-020.xgtf"
IMAGE = "..\\FIGC\\Sequence{}"
TRAIN_DIR = "..\\FIGC\\TrainDataset"
MAIN_DIR = "..\\FIGC"
sequences_train = {"1":{"flip":False},"3":{"flip":False},"4":{"flip":False}}
sequences_test = {"5":{"flip": False}, "6":{"flip": True}}
TEST_DIR = "..\\FIGC\\TestDataset"


In [ ]:
class Dataset:
    def __init__(self, sequences, augment=True):
        self.ns = {"base":"http://lamp.cfar.umd.edu/viper#",
                         "data":"http://lamp.cfar.umd.edu/viperdata#"}
        self.images = []
        self.imageopts = []
        self.augment = augment
        self.labels = None
        for seq,opt in sequences.items():
            curr_images = os.listdir(IMAGE.format(seq))
            seq_size = len(curr_images)
            self.images.extend([os.path.join(IMAGE.format(seq), i) for i in curr_images])
            self.imageopts.extend([opt] * seq_size)
            annotation_file = ANNOT.format(seq)
            tree = ET.parse(annotation_file)

            root = tree.getroot()
            Ball = root.find(".//base:object[@name='BALL']", self.ns)
            frames = int(Ball.attrib['framespan'].split(':')[1])
            curr_labels = np.full((seq_size, 2), -1, dtype=np.int16)
            BallPos = Ball.find("./*[@name='BallPos']")
            for datapoint in BallPos.findall('data:point', self.ns):
                frameno = int(datapoint.get('framespan').split(':')[0])
                if frameno > seq_size + 7:
                    break
                x,y = int(datapoint.get('x')), int(datapoint.get('y'))
                curr_labels[frameno-1-7] = np.array([x,y])

            if self.labels is None:
                self.labels = curr_labels
            else:
                self.labels = np.concatenate((self.labels, curr_labels), axis=0)

        assert len(self.images) == self.labels.shape[0]
        assert len(self.imageopts) == self.labels.shape[0]

    def generate_batches(self, batch_size=16):
        set_size = len(self.images)
        image_batch = []
        map_batch = []
        for i in range(set_size):
            y_true = np.concatenate((np.zeros((68,120,1), dtype=np.float32), np.ones((68,120,1), dtype=np.float32)), axis=-1)
            img = cv2.imread(self.images[i], 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            H,W,_ = img.shape
            if H < 1088 or W < 1920:
                img = np.pad(img, ((0,1088-H),(0,1920-W),(0,0)), 'constant', constant_values=0)
            if self.imageopts[i].get("flip") == True:
                img = cv2.flip(img, 1)

            x,y = self.labels[i]
            visible = False if x < 0 else True

            x,y = int(math.floor((int(x)-1)/16)), int(math.floor((int(y)-1)/16))
            if visible:
                # Label ground truth location with ball confidence 1 and bg confidence 0
                y_true[y,x,0] = 1
                y_true[y,x,1] = 0
                # Expand background confidence to neighbours
                bg_kernel = np.ones((3,3), dtype=np.uint8)
                y_true[:,:,1] = cv2.erode(y_true[:,:,1], bg_kernel, iterations=1)

            # Input images being resized to 480 by 272 to improve performance
            image_batch.append(cv2.resize(img, dsize=(480,272)))
            map_batch.append(y_true)
            if len(image_batch) == batch_size:
                yield (np.asarray(image_batch), np.asarray(map_batch))

                if self.augment:
                    # Image Augmentation
                    seq = iaa.Sequential([
                        iaa.OneOf([
                            iaa.MultiplyBrightness((0.5,1.5)),
                            iaa.MultiplyHue((0.5,1.5)),
                            iaa.MultiplySaturation((0.5,1.5)),
                            iaa.LinearContrast((0.4,1.6))
                        ]),
                        iaa.Fliplr(0.5),
                        iaa.Sometimes(0.50, iaa.Crop(percent=(0.0,0.25)))
                    ])
                    images_aug, maps_aug = seq(images=image_batch, heatmaps=map_batch)

                    yield (np.asarray(images_aug), np.asarray(maps_aug))

                image_batch.clear()
                map_batch.clear()
        if image_batch:
            yield (np.asarray(image_batch), np.asarray(map_batch))

            if self.augment:
                # Image Augmentation
                seq = iaa.Sequential([
                        iaa.OneOf([
                            iaa.MultiplyBrightness((0.5,1.5)),
                            iaa.MultiplyHue((0.5,1.5)),
                            iaa.MultiplySaturation((0.5,1.5)),
                            iaa.LinearContrast((0.4,1.6))
                        ]),
                        iaa.Fliplr(0.5),
                        iaa.Sometimes(0.50, iaa.Crop(percent=(0.0,0.33)))
                    ])
                images_aug, maps_aug = seq(images=image_batch, heatmaps=map_batch)

                yield (np.asarray(images_aug), np.asarray(maps_aug))

            del image_batch, map_batch

def load(DIR):
    def gen_dataset():
        file_list = os.listdir(DIR)
        # Infinitely iterable as Keras requires
        while True:
            for file in file_list:
                with open(os.path.join(DIR, file), 'rb') as f:
                    x_train, y_train = pkl.load(f)
                    yield (x_train, y_train)
    return gen_dataset


In [ ]:
def deepball_loss_function(y_true, y_pred):
    # y_true (batch_size, 68, 120, 2)
    # y_pred (batch_size, 68, 120, 2)

    ball_gt, bg_gt = y_true[:,:,:,0], y_true[:,:,:,1]
    N = K.sum(ball_gt, axis=(1,2)) + 1
    M = K.sum(bg_gt, axis=(1,2)) + 1
    zer = K.zeros_like(ball_gt)

    y_pred = K.log(y_pred)
    ball_cm = y_pred[:,:,:,0]
    bg_cm = y_pred[:,:,:,1]

    Lpos = K.sum(zer + (ball_cm * ball_gt), axis=(1,2))
    Lpos = K.sum(K.zeros_like(N) + (Lpos / tf.maximum(1.0, N)))

    Lneg = K.sum(zer + (bg_cm * bg_gt), axis=(1,2))
    Lneg = K.sum(K.zeros_like(M) + (Lneg / tf.maximum(1.0, M)))
#     print(K.eval(Lpos),K.eval(Lneg))

    # Multiplying by batch_size as Keras automatically averages the scalar output over it
    return (-Lpos - 0.3*Lneg) * 16

In [ ]:
def deepball_precision(y_true, y_pred):
    ball_gt = y_true[:,:,:,0]
    ball_cm = y_pred[:,:,:,0]

    thre_ball_cm = K.cast(K.greater(ball_cm, 0.998), "float32")
    tp = K.sum(ball_gt * thre_ball_cm)
    totalp = K.sum(K.max(thre_ball_cm, axis=(1,2)))

    return tp/tf.maximum(1.0, totalp)

In [ ]:
def peak_fp_confidence(y_true, y_pred):
    ball_gt = y_true[:,:,:,0]
    ball_cm = y_pred[:,:,:,0]
    N = 1 - K.sum(ball_gt, axis=(1,2))
    num_fp = K.sum(N)
    peak_c = K.max(ball_cm, axis=(1,2))

    return K.sum((N * peak_c)) / tf.maximum(1.0, num_fp)

In [ ]:
def _conv_block(inp, convs, pool=True):
    x = inp
    count = 0
    for conv in convs:
        count += 1
        x = Conv2D(conv['filter'],
                   conv['kernel'],
                   strides=conv['stride'],
                   padding='same',
                   use_bias=False)(x)
        x = BatchNormalization(epsilon=0.001)(x)
        x = ReLU()(x)
    if pool:
        x = MaxPool2D(pool_size=2)(x)
    return x

def get_functional_deepball_model():
    #Input image
    input_image = Input(shape=(272, 480, 3))

    # Conv1
    x = _conv_block(input_image, [{'filter': 8, 'kernel': 7, 'stride': 2, 'layer_idx': 0},
                                  {'filter': 8, 'kernel': 3, 'stride': 1, 'layer_idx': 1}])
    conv1 = x
    # Conv2
    x = _conv_block(x, [{'filter': 16, 'kernel': 3, 'stride': 1, 'layer_idx': 2},
                        {'filter': 16, 'kernel': 3, 'stride': 1, 'layer_idx': 3}])
    conv2 = x

    # Conv3
    x = _conv_block(x, [{'filter': 32, 'kernel': 3, 'stride': 1, 'layer_idx': 4},
                        {'filter': 32, 'kernel': 3, 'stride': 1, 'layer_idx': 5}])
    conv3 = x

    # Upsampling Conv2
    upsampled_conv2 = UpSampling2D(size=(2,2))(conv2)

    # Upsampling Conv3
    upsampled_conv3 = UpSampling2D(size=(4,4))(conv3)

    # Concatenation along channels axis
    concat = Concatenate(axis=-1)([conv1, upsampled_conv2, upsampled_conv3])
    x = concat

    # Conv4
    x = _conv_block(x, [{'filter': 56, 'kernel': 3, 'stride': 1, 'layer_idx':6},
                        {'filter': 2, 'kernel': 3, 'stride': 1, 'layer_idx': 7}], pool=False)

#     conv3c = Reshape((32*30*17,))(conv3)
#     cout = Dense(1, activation='sigmoid')(Dense(200, activation='relu')(conv3c))

    x = Softmax(axis=-1)(x)
    model = keras.Model(inputs=input_image, outputs=x)
    print(model.summary())
    return model

In [ ]:
get_functional_deepball_model()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 272, 480, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 136, 240, 8)    │          1,176 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 136, 240, 8)    │             32 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 136, 240, 8)    │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 136, 240, 8)    │            576 │ re_lu[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 136, 240, 8)    │             32 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1 (ReLU)            │ (None, 136, 240, 8)    │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 68, 120, 8)     │              0 │ re_lu_1[0][0]          │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 68, 120, 16)    │          1,152 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 68, 120, 16)    │             64 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_2 (ReLU)            │ (None, 68, 120, 16)    │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 68, 120, 16)    │          2,304 │ re_lu_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 68, 120, 16)    │             64 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_3 (ReLU)            │ (None, 68, 120, 16)    │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 34, 60, 16)     │              0 │ re_lu_3[0][0]          │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 34, 60, 32)     │          4,608 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4

 Total params: 48,944 (191.19 KB)

 Trainable params: 48,604 (189.86 KB)

 Non-trainable params: 340 (1.33 KB)

None


<Functional name=functional, built=True>

In [ ]:
def create_batches(dataset, DIR):
    d_generator = dataset.generate_batches(batch_size=16)
    batchno = 1
    for x_train, y_train in d_generator:
        with open(os.path.join(DIR, '{}.pkl'.format(batchno)), 'wb') as f:
            pkl.dump([x_train, y_train], f)
        print('saved to disk: {}'.format(batchno))
        batchno+=1

def get_compiled_deepball_model():
    # define the model
    model = get_functional_deepball_model()
#     model.summary()
    model.compile(optimizer='adam', loss=deepball_loss_function,
                  metrics=[deepball_precision])
    return model

def train(model):
    train_dataset = tf.data.Dataset.from_generator(load(TRAIN_DIR),
                                                    (tf.uint8, tf.uint8),
                                                    (tf.TensorShape([None,272,480,3]),tf.TensorShape([None,68,120,2]))).unbatch().map(lambda *args: [tf.cast(elem, tf.float32) for elem in args]).batch(16)
    model.fit(train_dataset, epochs=75, verbose=2, steps_per_epoch=1126)

    model.save_weights('deepball_weights.h5')
    keras.models.save_model(model, "deepballlocal.h5", save_format="h5")


In [ ]:
customObj = {'deepball_loss_function': deepball_loss_function, 'deepball_precision': deepball_precision}
testmodel = keras.models.load_model('/content/deepballlocal.h5', custom_objects=customObj)
# evaluation = testmodel.evaluate_generator(test_load, steps=375, verbose=1)
# print(evaluation)


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [ ]:
def testNet(model):
    sampimg = cv2.imread(os.path.join(MAIN_DIR, "Sequence6", "0010.bmp"), 1)
    sampimg = cv2.cvtColor(sampimg, cv2.COLOR_BGR2RGB)
    H,W,_ = sampimg.shape

    cmap = model.predict(np.array([cv2.resize(sampimg, (480,272))]), batch_size=1, verbose=1)
#     cmap = K.exp(cmap)

    ball_cm = cmap[0,:,:,0]
    bg_cm = cmap[0,:,:,1]

    pos = np.unravel_index(np.argmax(ball_cm, axis=None), ball_cm.shape)
    yp, xp = pos
    print(xp, yp)
    yp, xp = int(math.floor(16*yp)), int(math.floor(16*xp))
    print(xp, yp)
    print('Peak ball C : {}'.format(ball_cm[pos]))
    plt.figure(figsize=(8,10))
    plt.subplot(3,1,1)
    plt.imshow(bg_cm, cmap='gray')
    plt.subplot(3,1,2)
    plt.imshow(ball_cm, cmap='gray')
    plt.subplot(3,1,3)
    cv2.circle(sampimg, (xp,yp), 16, (43,0,255), thickness=2)
    plt.imshow(sampimg)
    plt.show()


In [ ]:
def display_ball_box(image, x, y):
    if x < 0:
        print("*** No ball detected ***")
    else:
        ball_pos = np.array([y,x])
        cv2.circle(image, (x, y), 16, (255,0,0), thickness=2)
    return image


In [ ]:
THRESHOLD_VAL = 0.999999
def ballTrack(cap, output, model):
    trackers = []
    i = 0
    frame_rate_divider = 1
#     bpos = deque()
    while(cap.isOpened()):
        stime = time.time()
        ret, frame = cap.read()
        if ret:
            if i % frame_rate_divider == 0:
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                (h, w) = frame.shape[:2]
                ky, kx = 4 * h/272.0, 4 * w/480.0
                cmap = model.predict(np.array([cv2.resize(frame.astype(np.float32), (480,272))]), batch_size=1, verbose=1)
                cm = cmap[0,:,:,0]
                pos = np.unravel_index(np.argmax(cm, axis=None), cm.shape)
                y,x = pos
                print(cm[y,x])
                x = -1 if cm[y,x] < THRESHOLD_VAL else x
                y,x = math.floor(ky * y), math.floor(kx * x)

                frame = display_ball_box(frame, x, y)
#                 bpos.append((y,x))
                output.write(frame)
                cv2_imshow(frame)
                i += 1

            print('FPS {:.1f}'.format(1 / (time.time() - stime)))

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    output.release()
    cv2.destroyAllWindows()


In [ ]:
cap = cv2.VideoCapture('/content/tes.avi')
size = (
	int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
	int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
)
codec = cv2.VideoWriter_fourcc(*'DIVX')
output = cv2.VideoWriter('Seq6Ballco4.avi',codec,25.0,size)
# outputSeg = cv2.VideoWriter('Seq6Seg.avi', codec,25.0,size)


In [ ]:
ballTrack(cap,output,testmodel)

Output hidden; open in https://colab.research.google.com to view.

For example, here we download and display a PNG image of the Colab logo: